https://docs.python.org/3.4/library/multiprocessing.html

In [1]:
import sys
print(sys.version)
import pandas
print('pandas',pandas.__version__)
import numpy
print('numpy',numpy.__version__)
from multiprocessing import Process
from multiprocessing import Pool
import os
import time

3.6.7 |Anaconda, Inc.| (default, Oct 23 2018, 19:16:44) 
[GCC 7.3.0]
pandas 0.23.4
numpy 1.13.3


# introduction to concurrent processes

In [2]:
def proc_info(some_str):
    print('arg =',some_str) # string that was passed in
    print('\nmodule name:', __name__)
    print('parent process:', os.getppid())
    print('process id:', os.getpid())

def myfunc(a_name):
    """ this function calls another function """
    print('now in myfunc')
    print('process ID =',os.getpid())
    proc_info(a_name)
    print('end of myfunc')

before using multiprocessing, inspect the processes of a normal python execution

In [3]:
print('process ID =',os.getpid(),'\n')
myfunc('mary')

process ID = 860 

now in myfunc
process ID = 860
arg = mary

module name: __main__
parent process: 7
process id: 860
end of myfunc


# launch a separate (serial) process<BR> using multiprocessing module

In [4]:
if __name__ == '__main__':
    proc_info('main code') # display info about this process
    print('\ncreate process')
    p = Process(target=myfunc, args=('bob',)) # from main, call a function
    print('\nstart process')
    p.start()
    print('\njoin')

    p.join()

arg = main code

module name: __main__
parent process: 7
process id: 860

create process

start process
now in myfunc
process ID = 878
arg = bob

module name:
join
 __main__
parent process: 860
process id: 878
end of myfunc


# multiprocessing for concurrent operations
Now that we've explored having multiple processes, use the processes concurrently

https://docs.python.org/3.4/library/multiprocessing.html#using-a-pool-of-workers

In [5]:
from multiprocessing import Pool

## _example_: map function to list

In [6]:
def sq_fun(x):
    return x*x

In [7]:
%%time

if __name__ == '__main__': # only the parent thread runs the pool
    
    with Pool(5) as p:
        print(p.map(sq_fun, [1, 2, 3]))

[1, 4, 9]
CPU times: user 210 ms, sys: 450 ms, total: 660 ms
Wall time: 1.2 s


## _example_: dataframe operation

Create random dataframe

https://stackoverflow.com/questions/32752292/how-to-create-a-data-frame-of-random-integers-with-pandas

In [8]:
df = pandas.DataFrame(
    numpy.random.randint(0,100,size=(100, 4)), 
    columns=list('ABCD')
    )

In [9]:
print('df shape:',df.shape)
df.head()

df shape: (100, 4)


A   B   C   D
0  41  83  56  97
1  12  29  48  68
2  14  70  71  47
3  83  40  25  74
4  64  98  14  12

http://www.racketracer.com/2016/07/06/pandas-in-parallel/

In [10]:
num_partitions = 7 #number of partitions to split dataframe
num_cores = 4 #number of cores on your machine

In [11]:
def parallelize_dataframe(df, func,num_cores):
    """
    Apply a function to chunks of a dataframe concurrently
    """
    df_split = numpy.array_split(df, num_partitions)
    with Pool(num_cores) as pool:
        df = pandas.concat(pool.map(func, df_split))
    return df

In [12]:
def multiply_columns(data):
    data['K'] = data['A'].apply(lambda x: x*2)
    return data

In [22]:
%time df_modified = parallelize_dataframe(df, multiply_columns,num_cores)

CPU times: user 50 ms, sys: 90 ms, total: 140 ms
Wall time: 463 ms


In [14]:
df_modified.head()

A   B   C   D    K
0  41  83  56  97   82
1  12  29  48  68   24
2  14  70  71  47   28
3  83  40  25  74  166
4  64  98  14  12  128

In [23]:
%time df['K'] = df['A'].apply(lambda x: x*2)

CPU times: user 0 ns, sys: 10 ms, total: 10 ms
Wall time: 10.6 ms


Applying the function to chunks took much more time! 

_Reason_: `multiprocessing` has to serialize the data, send the serialized data to each thread, then apply the transformation, then send the data back to main

In [16]:
import multiprocessing
import time

In [17]:
def serial_func(arg1):
    return arg1*2

In [18]:
%time list(map(serial_func,[3,4,5]))

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 24.3 µs


[6, 8, 10]

In [19]:
if __name__ == '__main__':
    start_time=time.time()
    res_list=[]
    with multiprocessing.Pool(processes=multiprocessing.cpu_count()) as pool:   # start worker processes
        res_list = pool.map(serial_func,[3,4,5]) # see https://docs.python.org/3/library/multiprocessing.html

    print('main elapsed:',time.time()-start_time,'seconds')

main elapsed: 0.5992202758789062 seconds


In [20]:
res_list

[6, 8, 10]